In [189]:
import sys
import numpy as np
import pickle
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import skimage.measure
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import time
import os
import copy
import string
char_list = list(" '" + string.ascii_lowercase + '12 ')

def absmax(nd_array):
    a = np.max(np.abs(nd_array))
    return(np.array([-a,a]))

def elapsed_time(t_start,unit):
    t_end = time.time()
    d = t_end - t_start
    if(unit=='min'):
        d /= 60
    elif(unit=='h'):
        d /= 3600
    print('%.2f '%d, unit + ' elapsed',sep='')

In [5]:
data_dir = "/data/asr_introspection/"

with open(data_dir + "vocabularies.pkl", "rb") as input_file:
    graphemes_list, phonemes_list, _ = pickle.load(input_file)
joint_list = graphemes_list + phonemes_list

with open("/data/asr_introspection/w2l_weights.pkl", "rb") as input_file:
    weights = pickle.load(input_file)
with open("/data/asr_introspection/w2l_bn_params.pkl", "rb") as input_file:
    bn_params = pickle.load(input_file)

In [3]:
# compute averages from one or multiple files
with open(data_dir + 'summation_abs_actxgrad_all_noblank.pkl', 'rb') as input_file:
    summation, n_samples = pickle.load(input_file)

# normalizer (average acts/grads over all characters (only chars!))
# should be similar to the blank label results <- check that
normalizer = copy.deepcopy(summation)
for layer_id in range(12):
    for value_set in ['acts','grads']:
        normalizer[layer_id][value_set] = np.sum(summation[layer_id][value_set][:27,:,:],0)/np.sum(n_samples[:27])

# average for each character/phoneme and both for acts and grads
averages = copy.deepcopy(summation)
normalized_averages = copy.deepcopy(summation)
for layer_id in range(12):
    for char_id in range(67):
        if(n_samples[char_id]>0):
            for value_set in ['acts','grads']:
                averages[layer_id][value_set][char_id,:,:] = summation[layer_id][value_set][char_id,:,:]/n_samples[char_id]
                normalized_averages[layer_id][value_set][char_id,:,:] = averages[layer_id][value_set][char_id,:,:] - normalizer[layer_id][value_set]

In [4]:
layer_ranges = [206,80,74,68,62,56,50,44,38,32,1,1,1]
n_filters = [256,256,256,256,256,256,256,256,256,2048,2048,31]
receptive_field_sizes = [48,60,72,84,96,108,120,132,144,206,206,206]
kernel_sizes = [48,7,7,7,7,7,7,7,7,32,1,1]
strides = [2,1,1,1,1,1,1,1,1,1,1,1]
GRIDS = {16: (4, 4), 32: (8, 4), 64: (8, 8), 128: (16, 8), 256: (16, 16),
         512: (32, 16), 1024: (32, 32), 2048: (64, 32), 31: (31,1)}

## visualize how filters look like, when neigborhood constraint is applied

- all filters as huge image
- close-up of a particular region
- each n-th filter in each dim as a smaller filter


In [34]:
np.max(filters)

2.5806406

In [412]:
filters = weights[0]
filter_grid_size = GRIDS[filters.shape[2]]
spacing = 0
filter_plot_mat = np.zeros((filter_grid_size[0]*(filters.shape[1]+spacing),
                            filter_grid_size[1]*(filters.shape[0]+spacing)))
for oc in range(filters.shape[2]):
    x_start = np.int(oc/filter_grid_size[1]) * (filters.shape[1]+spacing)
    y_start = oc%filter_grid_size[0] * (filters.shape[0]+spacing)
    filter_plot_mat[x_start:x_start+filters.shape[1],y_start:y_start+filters.shape[0]] = np.transpose(filters[:,:,oc])[::-1,:]
    
plt.figure(figsize=(20,60))
plt.imshow(filter_plot_mat,cmap='bwr')#,aspect=filters.shape[0]/filters.shape[1])
plt.clim(absmax(filter_plot_mat)/3)
plt.xticks(np.arange(filters.shape[0],filter_plot_mat.shape[1],filters.shape[0]),'')
plt.yticks(np.arange(filters.shape[1],filter_plot_mat.shape[0],filters.shape[1]),'')
plt.grid(color='black')
# plt.show()
plt.savefig(figure_dir+"regularized_filtermap.png",
                    dpi=100,
                    format='png',
                    bbox_inches='tight')
plt.close()

#close-up
subcoords = np.stack((np.array([1,4]) * (filters.shape[1]+spacing),
                      np.array([1,4]) * (filters.shape[0]+spacing)))
closeup_plot_mat = filter_plot_mat[subcoords[0,0]:subcoords[0,1]-spacing,
                           subcoords[1,0]:subcoords[1,1]-spacing]
plt.figure(figsize=(10,20))
plt.imshow(closeup_plot_mat,
           cmap='bwr')#,aspect=filters.shape[0]/filters.shape[1])
plt.clim(absmax(filter_plot_mat)/3)
plt.xticks(np.arange(filters.shape[0],closeup_plot_mat.shape[1],filters.shape[0]),'')
plt.yticks(np.arange(filters.shape[1],closeup_plot_mat.shape[0],filters.shape[1]),'')
plt.grid(color='black')
plt.title('x:'+str(subcoords[0,0])+'-'+str(subcoords[0,1])+' ; y:'+str(subcoords[1,0])+'-'+str(subcoords[1,1]))
# plt.show()
plt.savefig(figure_dir+"regularized_filtermap_closeup_1.pdf",
                    dpi=200,
                    format='pdf',
                    bbox_inches='tight')
plt.close()

subcoords = np.stack((np.array([12,15]) * (filters.shape[1]+spacing),
                      np.array([12,15]) * (filters.shape[0]+spacing)))
closeup_plot_mat = filter_plot_mat[subcoords[0,0]:subcoords[0,1]-spacing,
                           subcoords[1,0]:subcoords[1,1]-spacing]
plt.figure(figsize=(10,20))
plt.imshow(closeup_plot_mat,
           cmap='bwr')#,aspect=filters.shape[0]/filters.shape[1])
plt.clim(absmax(filter_plot_mat)/3)
plt.xticks(np.arange(filters.shape[0],closeup_plot_mat.shape[1],filters.shape[0]),'')
plt.yticks(np.arange(filters.shape[1],closeup_plot_mat.shape[0],filters.shape[1]),'')
plt.grid(color='black')
plt.title('x:'+str(subcoords[0,0])+'-'+str(subcoords[0,1])+' ; y:'+str(subcoords[1,0])+'-'+str(subcoords[1,1]))
# plt.show()
plt.savefig(figure_dir+"regularized_filtermap_closeup_2.pdf",
                    dpi=200,
                    format='pdf',
                    bbox_inches='tight')
plt.close()

## grid-shaped plot of GradNAP AUCs
- for different phonemes
- apply avg pooling

In [139]:
#from https://stackoverflow.com/questions/42463172/how-to-perform-max-mean-pooling-on-a-2d-array-using-numpy

def asStride(arr,sub_shape,stride):
    '''Get a strided sub-matrices view of an ndarray.
    See also skimage.util.shape.view_as_windows()
    '''
    s0,s1=arr.strides[:2]
    m1,n1=arr.shape[:2]
    m2,n2=sub_shape
    view_shape=(1+(m1-m2)//stride[0],1+(n1-n2)//stride[1],m2,n2)+arr.shape[2:]
    strides=(stride[0]*s0,stride[1]*s1,s0,s1)+arr.strides[2:]
    subs=np.lib.stride_tricks.as_strided(arr,view_shape,strides=strides)
    return subs

def poolingOverlap(mat,ksize,stride=None,method='max',pad=False):
    '''Overlapping pooling on 2D or 3D data.

    <mat>: ndarray, input array to pool.
    <ksize>: tuple of 2, kernel size in (ky, kx).
    <stride>: tuple of 2 or None, stride of pooling window.
              If None, same as <ksize> (non-overlapping pooling).
    <method>: str, 'max for max-pooling,
                   'mean' for mean-pooling.
    <pad>: bool, pad <mat> or not. If no pad, output has size
           (n-f)//s+1, n being <mat> size, f being kernel size, s stride.
           if pad, output has size ceil(n/s).

    Return <result>: pooled matrix.
    '''

    m, n = mat.shape[:2]
    ky,kx=ksize
    if stride is None:
        stride=(ky,kx)
    sy,sx=stride

    _ceil=lambda x,y: int(np.ceil(x/float(y)))

    if pad:
        ny=_ceil(m,sy)
        nx=_ceil(n,sx)
        size=((ny-1)*sy+ky, (nx-1)*sx+kx) + mat.shape[2:]
        mat_pad=np.full(size,np.nan)
        mat_pad[:m,:n,...]=mat
    else:
        mat_pad=mat[:(m-ky)//sy*sy+ky, :(n-kx)//sx*sx+kx, ...]

    view=asStride(mat_pad,ksize,stride)

    if method=='max':
        result=np.nanmax(view,axis=(2,3))
    else:
        result=np.nanmean(view,axis=(2,3))

    return result

In [119]:
joint_list[28]

'AA'

In [408]:
# layer_id = 1  # 0 would be the input
char_id = 29  # /AE/
layers = np.arange(1,12)
responsive_neuron_ids = [dict() for i in range(12)]

figure_dir = "/project/asr_introspection/figures/"
fig_format = "pdf"

for layer_id in layers:
    if(layer_id<10):
        fig = plt.figure(figsize=(6,9))
    else:
        fig = plt.figure(figsize=(6,18))
    grid = plt.GridSpec(3,2, hspace=0, wspace=0)
    
    for i, char_id in enumerate([28,29,58]):

        gradNAP = normalized_averages[layer_id]['acts'][char_id]

        AUC = np.sum(np.abs(gradNAP),1)
        AUC = np.sign(np.mean(gradNAP,1)) * AUC

#         absmax_sign = np.sign(np.max(gradNAP,1) + np.min(gradNAP,1))
#         AUC = absmax_sign * np.max(np.abs(gradNAP),1)
        
    
#         AUC_reshaped = np.reshape(AUC,GRIDS[AUC.shape[0]])
        AUC_reshaped = np.transpose(np.reshape(AUC,GRIDS[AUC.shape[0]][::-1]))
        

#         k_dimfactor = 4
#         ksize = (np.int(AUC_reshaped.shape[0]/k_dimfactor),
#                  np.int(AUC_reshaped.shape[1]/k_dimfactor))
        if(layer_id<10):    
            ksize = (3,3)
        else:
            ksize = (3,3)
        avg_pooled_AUC = poolingOverlap(mat = AUC_reshaped,
                                        ksize = ksize,
                                        stride = (1,1),
                                        method = 'mean',
                                        pad = False)
#         skimage.measure.block_reduce(AUC_reshaped, (3,3), np.mean)
        
        
        responsive_region_idx = np.argmax(np.abs(avg_pooled_AUC))
        responsive_x = responsive_region_idx%avg_pooled_AUC.shape[1]
        responsive_y = np.int(responsive_region_idx/avg_pooled_AUC.shape[1])
        
        
        helper_mat = copy.deepcopy(AUC_reshaped)
        helper_mat[responsive_y:responsive_y+ksize[1],responsive_x:responsive_x+ksize[0]] = np.max(helper_mat)+1
        neuron_ids = np.sort(np.argsort(np.concatenate(np.transpose(helper_mat)))[::-1][:np.prod(ksize)])
        neuron_ids = neuron_ids * np.sign(AUC[neuron_ids]).astype('int')
        responsive_neuron_ids[layer_id][joint_list[char_id]] = neuron_ids
        
        ax = fig.add_subplot(grid[i,0])
#         ax = plt.subplot(2,3,i+1)
        plt.imshow(AUC_reshaped,cmap='bwr')
        plt.clim(absmax(AUC))
        plt.xlim([-1.5,AUC_reshaped.shape[1]+0.5])
        plt.ylim([-1.5,AUC_reshaped.shape[0]+0.5])
        plt.axis('off')

        rect = mpl.patches.Rectangle((-0.5,-0.5),AUC_reshaped.shape[1],AUC_reshaped.shape[0],linewidth=1,edgecolor='#000000',facecolor='none')
        ax.add_patch(rect) 
        rect = mpl.patches.Rectangle((responsive_x-0.5,responsive_y-0.5),ksize[1],ksize[0],linewidth=3,edgecolor='#00ff00',facecolor='none')
        ax.add_patch(rect)  
        
        avg_pooled_AUC = np.pad(avg_pooled_AUC, 1, 'constant', constant_values=0)
        
        ax = fig.add_subplot(grid[i,1])
#         ax = plt.subplot(2,3,i+4)
        plt.imshow(avg_pooled_AUC,cmap='bwr')
        plt.clim(absmax(avg_pooled_AUC))
        plt.xlim([-1.5,avg_pooled_AUC.shape[1]+0.5])
        plt.ylim([-1.5,avg_pooled_AUC.shape[0]+0.5])
        plt.axis('off')
        
        rect = mpl.patches.Rectangle((0.5,0.5),avg_pooled_AUC.shape[1]-2,avg_pooled_AUC.shape[0]-2,linewidth=1,edgecolor='#000000',facecolor='none')
        ax.add_patch(rect) 
        rect = mpl.patches.Rectangle((responsive_x+0.5,responsive_y+0.5),1,1,linewidth=3,edgecolor='#00ff00',facecolor='none')
        ax.add_patch(rect)   
        
    plt.savefig(figure_dir + "responsive_regions_layer"+str(layer_id)+"."+fig_format,
                dpi=300,
                format=fig_format,
                bbox_inches='tight')
    plt.close()
    print("plot saved")
#     plt.show()


plot saved
plot saved
plot saved
plot saved
plot saved
plot saved
plot saved
plot saved
plot saved
plot saved
plot saved


## feature vis for active GradNAP AUC plot regions
- for different phonemes

In [374]:
def build_dict(layer_id, input_tensor, weights, bn_params):
    d = dict()
#     d[input_tensor]=[np.transpose(spec)]
    
    if layer_id==11:
        d['out/bias:0'] = weights[11]
        d['out/kernel:0'] = weights[12]
        layer_id -= 1
    
    for i in np.arange(layer_id+1):
        d['conv_'+str(i)+'/kernel:0'] = weights[i]
        d['batch_norm_'+str(i)+'/beta:0'] = bn_params[(i*4)]
        d['batch_norm_'+str(i)+'/gamma:0'] = bn_params[(i*4)+1]
        d['batch_norm_'+str(i)+'/moving_mean:0'] = bn_params[(i*4)+2]
        d['batch_norm_'+str(i)+'/moving_variance:0'] = bn_params[(i*4)+3]
    
    return d

def conv_layer(inputs, n_filter, kernel_size, stride, layer_id, out_layer=False):
    if(not out_layer):
        conv = tf.layers.conv1d(
            inputs = inputs, 
            filters = n_filter, 
            kernel_size = kernel_size,
            strides=stride, 
            activation=None,
            use_bias=False, 
            padding="valid",
            name="conv_"+str(layer_id),
            data_format="channels_last",
            trainable=False)

        conv_bn = tf.layers.batch_normalization(
            conv, 
            axis=2, 
            training=False, 
            name="batch_norm_"+str(layer_id),
            trainable=False)

        relu_out = tf.nn.relu(conv_bn)

        return(conv_bn,relu_out)
    else:
        conv = tf.layers.conv1d(
            inputs = inputs, 
            filters = n_filter, 
            kernel_size = kernel_size,
            strides=stride, 
            activation=None,
            use_bias=True, 
            padding="valid",
            name="out",
            data_format="channels_last",
            trainable=False)

        return(conv)

def build_w2l_model(weights,bn_params,layer_id,filter_ids):
    tf.reset_default_graph()

    n_filters = [256,256,256,256,256,256,256,256,256,2048,2048,31]
    kernel_sizes = [48,7,7,7,7,7,7,7,7,32,1,1]
    strides = [2,1,1,1,1,1,1,1,1,1,1,1]
    GRIDS = {16: (4, 4), 32: (8, 4), 64: (8, 8), 128: (16, 8), 256: (16, 16),
             512: (32, 16), 1024: (32, 32), 2048: (64, 32), 31: (31,1)}
    receptive_field_sizes = [48,60,72,84,96,108,120,132,144,206,206,206]
    
    x = tf.get_variable("x", [1,206,128], dtype=tf.float32,
                        initializer = tf.initializers.random_normal(mean=0,stddev=0.001))
#     x = 5 * (x / tf.reduce_max(tf.abs(x)) )  # spectrogram data is in the range [-4.98,4.64]
    
    x_perturbated = tf.concat([x for i in range(5)],0) #input robust to n random perturbations
    x_perturbated = x_perturbated + tf.random_normal(tf.shape(x_perturbated),mean=0,stddev=0.05)

    layer_outs = []
    pre_activations = []
    p,l = conv_layer(x_perturbated,n_filters[0],kernel_sizes[0],strides[0],0)
    pre_activations.append(p)
    layer_outs.append(l)

    for i in np.arange(11)[1:]:
        p,l = conv_layer(layer_outs[i-1],n_filters[i],kernel_sizes[i],strides[i],i)
        pre_activations.append(p)
        layer_outs.append(l)

    pre_activations.append(conv_layer(layer_outs[10],n_filters[11],kernel_sizes[11],strides[11],11,out_layer=True))
    layer_outs.append(tf.nn.softmax(pre_activations[11]))
    
#     loss = layer_outs[layer_id][:,tf.cast(tf.floor(tf.shape(layer_outs[layer_id])[1]/2),tf.int32),:]
#     loss = layer_outs[layer_id][:,0,:]
#     loss = tf.reduce_sum(np.sign(filter_ids)*(-1)*tf.reduce_sum(tf.gather(loss,np.abs(filter_ids),axis=1),axis=0))
    
    opt_target = pre_activations[layer_id][:,0,:]
    loss = (-1)*tf.reduce_sum(tf.gather(opt_target,np.abs(filter_ids),axis=1),axis=0)
    loss = tf.reduce_sum(np.sign(filter_ids)*loss)
    
    non_responsives_sum = tf.reduce_sum(tf.gather(tf.abs(opt_target),
                                                  np.setdiff1d(np.arange(n_filters[layer_id]),
                                                               np.abs(filter_ids)),
                                                  axis=1),
                                        axis=0)
    sparsity_loss = 0*tf.reduce_sum(tf.abs(non_responsives_sum))
    loss = loss + sparsity_loss
    
    
    
    # regularization parameters dependent on width of receptive field in the respective layer
    loss = loss + tf.contrib.layers.apply_regularization(
        regularizer=tf.contrib.layers.l1_l2_regularizer(15 / receptive_field_sizes[layer_id],
                                                        0.1 / receptive_field_sizes[layer_id]),
        weights_list=[x]
    )
    
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.05)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    
    return([train_op,x,loss,sparsity_loss,opt_target])


In [410]:
receptive_field_sizes = [48,60,72,84,96,108,120,132,144,206,206,206]
GRIDS = {16: (4, 4), 32: (8, 4), 64: (8, 8), 128: (16, 8), 256: (16, 16),
             512: (32, 16), 1024: (32, 32), 2048: (64, 32), 31: (31,1)}
n_steps = 16
print_each = 0#np.int(n_steps/5)
n_repetitions = 1
layers = np.arange(1,12)
character_ids = [28,29,58]#np.arange(len(joint_list))

# create region-wise optimal inputs
for layer_id in layers:
    fig = plt.figure(figsize=(5,15))
    grid = plt.GridSpec(3,1, hspace=0.1, wspace=0)
    for idx, char_id in enumerate(character_ids):
        # for neuron_id in range(len(responsive_neuron_ids[layer_id])):
        tf.reset_default_graph()
        char_responsive_neurons = responsive_neuron_ids[layer_id][joint_list[char_id]]
        ops = build_w2l_model(weights, bn_params, layer_id, char_responsive_neurons
        #                           [neuron_id:neuron_id+1]
                             )
        mean_optimal_input = np.zeros([1,206,128])
        with tf.Session() as sess:
            init = tf.global_variables_initializer()
            sess.run(init)

            d = build_dict(11,None,weights,bn_params)
            for i in range(n_steps):
                _, optimal_input,_,_, opt_target = sess.run(ops,feed_dict=d)
        
        ax = fig.add_subplot(grid[idx])
        plt.imshow(np.transpose(optimal_input[0,:receptive_field_sizes[layer_id],::-1]),cmap='bwr')
        plt.clim(absmax(optimal_input))
        plt.axis('off')
        rect = mpl.patches.Rectangle((-0.5,-0.5),receptive_field_sizes[layer_id],optimal_input.shape[2],linewidth=1,edgecolor='#000000',facecolor='none')
        ax.add_patch(rect) 
        
    plt.savefig(figure_dir + "joint_featurevis_layer"+str(layer_id)+"."+fig_format,
                dpi=300,
                format=fig_format,
                bbox_inches='tight')
    plt.close()
    print("plot saved")
#     plt.show()

plot saved
plot saved
plot saved
plot saved
plot saved
plot saved
plot saved
plot saved
plot saved
plot saved
plot saved


In [ ]:
receptive_field_sizes = [48,60,72,84,96,108,120,132,144,206,206,206]
GRIDS = {16: (4, 4), 32: (8, 4), 64: (8, 8), 128: (16, 8), 256: (16, 16),
             512: (32, 16), 1024: (32, 32), 2048: (64, 32), 31: (31,1)}
n_steps = 16
print_each = 0#np.int(n_steps/5)
n_repetitions = 1
layers = np.arange(1,12)
character_ids = [28,29,58]#np.arange(len(joint_list))

# create region-wise optimal inputs
for layer_id in layers:
    for idx, char_id in enumerate(character_ids):
        fig = plt.figure(figsize=(8,15))
        grid = plt.GridSpec(3,3, hspace=0.05, wspace=0.05,)
        
        for grid_idx, responsive_neuron in enumerate(responsive_neuron_ids[layer_id][joint_list[char_id]]):
            tf.reset_default_graph()
            ops = build_w2l_model(weights, bn_params, layer_id, [responsive_neuron])
            mean_optimal_input = np.zeros([1,206,128])
            with tf.Session() as sess:
                init = tf.global_variables_initializer()
                sess.run(init)

                d = build_dict(11,None,weights,bn_params)
                for i in range(n_steps):
                    _, optimal_input,_,_, opt_target = sess.run(ops,feed_dict=d)

            ax = fig.add_subplot(grid[grid_idx%3,np.int(grid_idx/3)])
            plt.imshow(np.transpose(optimal_input[0,:receptive_field_sizes[layer_id],::-1]),cmap='bwr')
            plt.clim(absmax(optimal_input))
            plt.axis('off')
            rect = mpl.patches.Rectangle((-0.5,-0.5),receptive_field_sizes[layer_id],optimal_input.shape[2],linewidth=1,edgecolor='#000000',facecolor='none')
            ax.add_patch(rect) 

        plt.savefig(figure_dir + "single_featurevis_layer"+str(layer_id)+"_phoneme_"+joint_list[char_id]+"."+fig_format,
                    dpi=300,
                    format=fig_format,
                    bbox_inches='tight')
        plt.close()
        print("plot saved")
#         plt.show()

plot saved
plot saved
plot saved
plot saved
plot saved
plot saved
plot saved
plot saved
plot saved
plot saved
plot saved
plot saved
plot saved
plot saved
plot saved
plot saved
plot saved
plot saved
plot saved
plot saved
plot saved
plot saved
plot saved
plot saved
plot saved
plot saved
plot saved
plot saved
plot saved
plot saved
plot saved


In [391]:
# #longer set of different plots


# for layer_id in layers:
#     for char_id in character_ids:
#         # for neuron_id in range(len(responsive_neuron_ids[layer_id])):
#         tf.reset_default_graph()
#         char_responsive_neurons = responsive_neuron_ids[layer_id][joint_list[char_id]]
#         ops = build_w2l_model(weights, bn_params, layer_id, char_responsive_neurons
#         #                           [neuron_id:neuron_id+1]
#                              )
#         mean_optimal_input = np.zeros([1,206,128])
#         with tf.Session() as sess:
#             init = tf.global_variables_initializer()
#             sess.run(init)

#             d = build_dict(11,None,weights,bn_params)
#             loss_curve = list()
#             sparsity_loss_curve = list()
#             for i in range(n_steps):
#                 _, optimal_input,tracking_loss,sparsity_loss, opt_target = sess.run(ops,feed_dict=d)
#                 loss_curve.append(tracking_loss)
#                 sparsity_loss_curve.append(sparsity_loss)

#         # plt.plot(np.array(loss_curve),'-')
#         # plt.show()

#         # plt.plot(np.array(loss_curve)-np.array(sparsity_loss_curve),'-')
#         # plt.show()

#         # plt.plot(np.array(sparsity_loss_curve),'-')
#         # plt.show()

#         plt.imshow(np.transpose(optimal_input[0,:receptive_field_sizes[layer_id],::-1]),cmap='bwr')
#         plt.clim(absmax(optimal_input))
#         plt.show()

#         responsive_neurons_mat = np.repeat(0,opt_target.shape[1])
#         responsive_neurons_mat[np.abs(char_responsive_neurons
#         #                                   [neuron_id:neuron_id+1]
#                                      )] = np.sign(char_responsive_neurons
#         #                                               [neuron_id:neuron_id+1]
#                                                  )
# #         plt.imshow(np.transpose(np.reshape(responsive_neurons_mat,GRIDS[opt_target.shape[1]][::-1])),cmap='bwr')
# #         plt.clim(absmax(responsive_neurons_mat))
# #         plt.xticks(np.arange(GRIDS[opt_target.shape[1]][0])-0.5,"")
# #         plt.yticks(np.arange(GRIDS[opt_target.shape[1]][1])-0.5,"")
# #         plt.grid()
# #         plt.show()

# #         plt.imshow(np.transpose(np.reshape(opt_target[0,:],GRIDS[opt_target.shape[1]][::-1])),cmap='bwr')
# #         plt.clim(absmax(opt_target))
# #         plt.xticks(np.arange(GRIDS[opt_target.shape[1]][0])-0.5,"")
# #         plt.yticks(np.arange(GRIDS[opt_target.shape[1]][1])-0.5,"")
# #         plt.grid()
# #         plt.show()